In [1]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split,GridSearchCV

housing=fetch_california_housing()
print(housing)

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]], shape=(20640, 8)), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset

In [10]:
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 

In [3]:
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data['price']=housing.target
data.head(10)

MedInc  HouseAge  AveRooms  AveBedrms  ...  AveOccup  Latitude  Longitude  price
0  8.3252      41.0  6.984127   1.023810  ...  2.555556     37.88    -122.23  4.526
1  8.3014      21.0  6.238137   0.971880  ...  2.109842     37.86    -122.22  3.585
2  7.2574      52.0  8.288136   1.073446  ...  2.802260     37.85    -122.24  3.521
3  5.6431      52.0  5.817352   1.073059  ...  2.547945     37.85    -122.25  3.413
4  3.8462      52.0  6.281853   1.081081  ...  2.181467     37.85    -122.25  3.422
5  4.0368      52.0  4.761658   1.103627  ...  2.139896     37.85    -122.25  2.697
6  3.6591      52.0  4.931907   0.951362  ...  2.128405     37.84    -122.25  2.992
7  3.1200      52.0  4.797527   1.061824  ...  1.788253     37.84    -122.25  2.414
8  2.0804      42.0  4.294118   1.117647  ...  2.026891     37.84    -122.26  2.267
9  3.6912      52.0  4.970588   0.990196  ...  2.172269     37.84    -122.25  2.611

[10 rows x 9 columns]

In [ ]:
## Train test split, model hyperparameter tuning, MLFLOW Experiments

from urllib.parse import urlparse
## independent and dependent features
x=data.drop(columns=["price"])
y=data["price"]




In [6]:
## Hyperparameter tuning usning grid search CV
def hyperparameter_tuning(x_train,y_train,param_grid):
    rf=RandomForestRegressor()
    grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=3,n_jobs=-1,verbose=2,scoring="neg_mean_squared_error")
    grid_search.fit(x_train,y_train)
    return grid_search

In [7]:
## split data into training and test sets

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)
from mlflow.models import infer_signature
import mlflow.sklearn
import mlflow.sklearn
signature=infer_signature(x_train,y_train)

## defne the hyperparameter grid

param_grid = {
    'n_estimators': [100, 200],
    'max_depth' : [5, 10, None],
    'min_samples_split' : [2,5],
    'min_samples_leaf' : [1,2]

}   

## start the mlflow experiments

with mlflow.start_run():
    ## perform hyperparameter tuning
    grid_search=hyperparameter_tuning(x_train,y_train,param_grid)

    ## get the best model 
    best_model=grid_search.best_estimator_

    ## evaluate the best model
    y_pred=best_model.predict(x_test)
    mse=mean_squared_error(y_test,y_pred)

    ## log best parameters and metrics
    mlflow.log_param("best_n_estimators",grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth",grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split",grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf",grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse",mse)

    ## Tracking uri
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store !='file':
        mlflow.sklearn.log_model(best_model,"model",registered_model_name="Best Randomforest Model")
    else:
        mlflow.sklearn.load_model(best_model,"model",signature=signature)
    
    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse}")



Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/06/03 13:40:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best Randomforest Model'.
2025/06/03 13:40:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best Randomforest Model, version 1


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Mean Squared Error: 0.2595727108865432
🏃 View run dashing-crane-798 at: http://127.0.0.1:5000/#/experiments/0/runs/14f9d645649a4999b6d5bcdfeb6391b5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'Best Randomforest Model'.
